<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Finding-forced-trends-in-oceanic-oxygen" data-toc-modified-id="Finding-forced-trends-in-oceanic-oxygen-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Finding forced trends in oceanic oxygen</a></span><ul class="toc-item"><li><span><a href="#Learning-Objectives" data-toc-modified-id="Learning-Objectives-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Learning Objectives</a></span></li><li><span><a href="#Create-Dask-Cluster" data-toc-modified-id="Create-Dask-Cluster-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Create Dask Cluster</a></span></li><li><span><a href="#Open-an-intake-esm-catalog-for-CESM1-LE-data-stored-on-Glade-and-Tape" data-toc-modified-id="Open-an-intake-esm-catalog-for-CESM1-LE-data-stored-on-Glade-and-Tape-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Open an intake-esm catalog for CESM1-LE data stored on Glade and Tape</a></span></li><li><span><a href="#Load-oxygen-Data-From-Control-Run-into-an-Xarray-Dataset-and-Compute-its-Annual-Mean" data-toc-modified-id="Load-oxygen-Data-From-Control-Run-into-an-Xarray-Dataset-and-Compute-its-Annual-Mean-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Load oxygen Data From Control Run into an Xarray Dataset and Compute its Annual Mean</a></span></li><li><span><a href="#Load-oxygen-Data-From-20C,-RCP85-Runs-into-an-Xarray-Dataset-and-Compute-its-Annual-Mean" data-toc-modified-id="Load-oxygen-Data-From-20C,-RCP85-Runs-into-an-Xarray-Dataset-and-Compute-its-Annual-Mean-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Load oxygen Data From 20C, RCP85 Runs into an Xarray Dataset and Compute its Annual Mean</a></span></li><li><span><a href="#Compute-Linear-Trend" data-toc-modified-id="Compute-Linear-Trend-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Compute Linear Trend</a></span></li><li><span><a href="#Plot-All-Trends---Total" data-toc-modified-id="Plot-All-Trends---Total-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Plot All Trends - Total</a></span></li><li><span><a href="#Plot-All-Trends---Internal" data-toc-modified-id="Plot-All-Trends---Internal-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Plot All Trends - Internal</a></span></li><li><span><a href="#Going-Further" data-toc-modified-id="Going-Further-1.9"><span class="toc-item-num">1.9&nbsp;&nbsp;</span>Going Further</a></span></li></ul></li></ul></div>

# Finding forced trends in oceanic oxygen

## Learning Objectives


- 

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from ncar_jobqueue import NCARCluster
from dask.distributed import Client
import esmlab
import xarray as xr
import numpy as np
import dask
import util

## Create Dask Cluster

In [ ]:
cluster = NCARCluster(memory="100GB", cores=4, processes=1)
cluster.adapt(minimum=3, maximum=12,
              wait_count=60)  # Adaptively scale between 3 and 12 dask workers
client = Client(cluster)  # Connect this local process to remote workers
cluster

In [ ]:
client

(👆don't forget to look at the dashboard!)

## Open an intake-esm catalog for CESM1-LE data stored on Glade and Tape

In [ ]:
import intake
from intake_esm import config

In [ ]:
#with config.set({'database-directory': '/glade/u/home/abanihi/.intake_esm/collections'}):
col = intake.open_esm_metadatastore(collection_name='CESM1-LE')

col.df.head()

In [ ]:
keep_vars = ['O2', 'z_t', 'KMT', 'TAREA', 'TLONG',
             'TLAT', 'time', 'time_bound', 'member_id']
yr0, yr1 = 2006, 2055
time_slice = slice(f'{yr0:04d}', f'{yr1:04d}')
ctrl_time_slice = slice(f'{yr0-1448:04d}', f'{yr1-1448:04d}')

## Load oxygen Data From Control Run into an Xarray Dataset and Compute its Annual Mean

In [ ]:
%%time
dsets = col.search(experiment=['CTRL'], variable=['O2'])\
        .to_xarray(decode_times=False, chunks={'time': 240})
_, dd = dsets.popitem()
ctrl = util.clean_ds(dd, keep_vars)
ctrl = util.sel_time(ctrl, ctrl_time_slice)
ctrl = esmlab.resample(ctrl, freq='ann')
print(ctrl)

## Load oxygen Data From 20C, RCP85 Runs into an Xarray Dataset and Compute its Annual Mean

In [ ]:
%%time
dsets = col.search(experiment=['20C', 'RCP85'], variable=['O2'])\
        .to_xarray(decode_times=False, chunks={'time': 240})
print(dsets.keys())
ds_20c = dsets['ocn.20C.pop.h']
ds_rcp85 = dsets['ocn.RCP85.pop.h']
dd = xr.concat([ds_20c, ds_rcp85], dim='time')
ds = util.clean_ds(dd, keep_vars)
ds = util.sel_time(ds, time_slice)
ds = esmlab.resample(ds, freq='ann')
print(ds)

## Compute Linear Trend

Define helper functions for trend computation.

In [ ]:
def linear_trend(da, dim='time'):
    da_chunk = da.chunk({dim: -1})
    trend = xr.apply_ufunc(calc_slope,
                           da_chunk,
                           vectorize=True,
                           input_core_dims=[[dim]],
                           output_core_dims=[[]],
                           output_dtypes=[np.float],
                           dask='parallelized')
    return trend


def calc_slope(y):
    """ufunc to be used by linear_trend"""
    x = np.arange(len(y))
    return np.polyfit(x, y, 1)[0]

Compute linear trends on the control integration.

In [ ]:
%%time
ctrl_trend = linear_trend(ctrl.O2.chunk({
    'nlat': 30,
    'nlon': 30,
    'time': -1
})).load()
ctrl_trend = ctrl_trend * len(ctrl.time)
ctrl_trend.attrs['units'] = f'mmol m$^{{-3}}$ ({len(ctrl.time)} yr)$^{{-1}}$'
ctrl_trend.plot()

Compute linear trends on the full ensemble.

In [ ]:
%%time
ds_trend = linear_trend(ds.O2.chunk({'nlat': 30, 'nlon': 30, 'time': -1}))\
                       .load()
ds_trend = ds_trend * len(ds.time)
ds_trend.attrs['units'] = f'mmol m$^{{-3}}$ ({len(ds.time)} yr)$^{{-1}}$'
ds_trend.isel(member_id=3).plot()

## Visualize trends

For visualization purposes, look for the ensemble members with the smallest and largest regional trends. In this case, we'll focus on the N. Pacific.

First step: compute the mean trend over the N. Pacific.

In [ ]:
npac_trend = ds_trend.where((10 < ds.TLAT) & (ds.TLAT < 65) & (120 < ds.TLONG)
                            & (ds.TLONG < 260))
npac_trend = esmlab.weighted_mean(npac_trend,
                                  dim=('nlat', 'nlon'),
                                  weights=ds.TAREA).load()

In [ ]:
npac_trend

Next, pick out the ensemble members with the min and max trends.

In [ ]:
member_id_pick = [
    npac_trend.where(npac_trend == npac_trend.max(),
                     drop=True).member_id.values.astype('int')[0],
    npac_trend.where(npac_trend == npac_trend.min(),
                     drop=True).member_id.values.astype('int')[0]
]

In [ ]:
member_id_pick

Import plotting modules

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import cmocean
import cartopy.crs as ccrs
import cartopy

Make a dataset suitable for plotting. In this case, `pop_add_cyclic` reformats the data on the ocean grid to enable seamless plotting on a contour map.

In [ ]:
dsp = util.pop_add_cyclic(
    xr.Dataset({
        'trend': ds_trend,
        'TLAT': ds.TLAT[0].compute(),
        'TLONG': ds.TLONG[0].compute()
    }))
dsp

Compute the `forced` (ensemble mean) and `internal` components. Call `compute` on these dataset to trigger loading into memory so that subsequent plotting operations proceed as rapidly as possible. 

In [ ]:
forced = dsp.trend.mean('member_id').compute()
internal = [(dsp.trend.sel(member_id=i) - forced).compute()
            for i in member_id_pick]

forced

Set up some basic plotting variables, i.e., contour levels, map projection.

In [ ]:
levels = np.arange(-100, 45, 5)
norm = colors.DivergingNorm(vmin=levels[0], vmax=levels[-1], vcenter=0.)

extent = [120, 260, 10, 65]
prj = ccrs.Mercator(central_longitude=np.mean(extent[0:2]),
                    min_latitude=extent[2],
                    max_latitude=extent[3])

Define a helper function that makes a single map plot.

In [ ]:
def one_plot(da, lines=True):
    # filled contours
    cf = ax.contourf(dsp.TLONG,
                     dsp.TLAT,
                     da,
                     levels=levels,
                     norm=norm,
                     cmap=cmocean.cm.curl_r,
                     extend='both',
                     transform=ccrs.PlateCarree())

    # contour lines
    cs = ax.contour(dsp.TLONG,
                    dsp.TLAT,
                    da,
                    colors='k',
                    levels=levels,
                    linewidths=0.5,
                    transform=ccrs.PlateCarree())

    if lines:
        # add contour labels
        lb = plt.clabel(cs, fontsize=6, inline=True, fmt='%r')

    # land
    land = ax.add_feature(
        cartopy.feature.NaturalEarthFeature('physical',
                                            'land',
                                            '110m',
                                            facecolor='lightgray'))

    ax.coastlines(linewidth=0.5)

    return cf

Make a plot showing the Total trend and it's internal and forced components for the ensemble members with the min and max N. Pacific trends.

In [ ]:
fig = plt.figure(figsize=(12, 6))

axs = []

# plot total
ax = fig.add_subplot(2, 3, 1, projection=prj)
ax.set_extent(extent)
one_plot(dsp.trend.sel(member_id=member_id_pick[0]))
ax.text(235., 60, f'{member_id_pick[0]:03d}', transform=ccrs.PlateCarree())
ax.set_title('Total')
axs.append(ax)

ax = fig.add_subplot(2, 3, 4, projection=prj)
ax.set_extent(extent)
one_plot(dsp.trend.sel(member_id=member_id_pick[1]))
ax.text(235., 60, f'{member_id_pick[1]:03d}', transform=ccrs.PlateCarree())
axs.append(ax)

# plot internal variability
ax = fig.add_subplot(2, 3, 2, projection=prj)
ax.set_extent(extent)
one_plot(internal[0])
ax.text(235., 60, f'{member_id_pick[0]:03d}', transform=ccrs.PlateCarree())
ax.set_title('Internal')
axs.append(ax)

ax = fig.add_subplot(2, 3, 5, projection=prj)
ax.set_extent(extent)
one_plot(internal[1])
ax.text(235., 60, f'{member_id_pick[1]:03d}', transform=ccrs.PlateCarree())
axs.append(ax)

# plot forced
ax = fig.add_subplot(2, 3, 3, projection=prj)
ax.set_extent(extent)
one_plot(forced)
ax.set_title('Forced')
axs.append(ax)

ax = fig.add_subplot(2, 3, 6, projection=prj)
ax.set_extent(extent)
cf = one_plot(forced)
axs.append(ax)

# add colorbar
plt.subplots_adjust(hspace=0.02, wspace=0.02)

# colorbar and labels
cb = plt.colorbar(cf, shrink=0.5, orientation='horizontal', ax=axs, pad=0.075)
cb.ax.set_title(dsp.trend.units)
plt.savefig('trend-decomp-O2-200m-NPac.png', dpi=300, bbox_inches='tight')

### Plot All Trends - Total

Make a postage stamp plot showing all the trends. First compute each ensemble member to load into memory.

In [ ]:
field = [
    dsp.trend.isel(member_id=i).compute() for i in range(0, len(ds.member_id))
]

In [ ]:
fig = plt.figure(figsize=(10, 10))

axs = []

# plot total
for i in range(0, len(ds.member_id)):
    ax = fig.add_subplot(7, 5, i + 1, projection=prj)
    ax.set_extent(extent)
    cf = one_plot(field[i], lines=False)
    axs.append(ax)

plt.subplots_adjust(hspace=0.04, wspace=0.02)

# colorbar and labels
cb = plt.colorbar(cf, shrink=0.5, orientation='horizontal', ax=axs, pad=0.075)
cb.ax.set_title(dsp.trend.units)

plt.savefig('all-trends-O2-200m-NPac.png', dpi=300, bbox_inches='tight')

### Plot All Trends - Internal

In [ ]:
fig = plt.figure(figsize=(10, 10))

axs = []

# plot total
for i in range(0, len(ds.member_id)):
    ax = fig.add_subplot(7, 5, i + 1, projection=prj)
    ax.set_extent(extent)
    cf = one_plot(field[i] - forced, lines=False)
    axs.append(ax)

plt.subplots_adjust(hspace=0.04, wspace=0.02)

# colorbar and labels
cb = plt.colorbar(cf, shrink=0.5, orientation='horizontal', ax=axs, pad=0.075)
cb.ax.set_title(dsp.trend.units)

plt.savefig('all-trends-internal-O2-200m-NPac.png',
            dpi=300,
            bbox_inches='tight')

## Going Further